# Create Merged Dataset

In this workbook, you will read in the `trip` and `fare` files. You are welcome to use DataFrame and/or SparkSQL API as you desire as long as it produces the expected results.

## Instructions:

1. Join both datasets such that you get a merged dataset with 21 unique fields. You need to determine how to join the dataset.
2. Once you create the merged dataset, you need to convert fields to the following types, since all fields were read is as string:
    * pickup_datetime and dropoff_datetime must be TIMESTAMP
    * passenger_count and rate_code must be INT
    * all other numeric fields must be FLOAT
    * the remaining fields stay as STRING
3. Save your merged and converted dataset to your own S3 bucket in parquet format.

You are welcome to add as many cells as you need below up until the next section. **You must include comments in your code.**

## Start Pyspark

In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("assign-4-merge").getOrCreate()

In [2]:
spark

## Load Data

In [3]:
trip = spark.read.parquet("s3://bigdatateaching/nyctaxi-2013/parquet/trip/")

In [4]:
fare = spark.read.parquet('s3://bigdatateaching/nyctaxi-2013/parquet/fare/')

## Explore Data

In [5]:
trip.printSchema()

root
 |-- medallion: string (nullable = true)
 |-- hack_license: string (nullable = true)
 |-- vendor_id: string (nullable = true)
 |-- rate_code: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- dropoff_datetime: string (nullable = true)
 |-- passenger_count: string (nullable = true)
 |-- trip_time_in_secs: string (nullable = true)
 |-- trip_distance: string (nullable = true)
 |-- pickup_longitude: string (nullable = true)
 |-- pickup_latitude: string (nullable = true)
 |-- dropoff_longitude: string (nullable = true)
 |-- dropoff_latitude: string (nullable = true)



In [6]:
fare.printSchema()

root
 |-- medallion: string (nullable = true)
 |-- hack_license: string (nullable = true)
 |-- vendor_id: string (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: string (nullable = true)
 |-- surcharge: string (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- tip_amount: string (nullable = true)
 |-- tolls_amount: string (nullable = true)
 |-- total_amount: string (nullable = true)



In [7]:
print('rows of trip:',trip.count())
print('rows of fare:',fare.count())

rows of trip: 173179759
rows of fare: 173179759


In [8]:
trip.take(2)

[Row(medallion='28EEFA38E24E9907C4667104BAAE17A2', hack_license='D27C32507727BFF647AF9272F7E3C651', vendor_id='CMT', rate_code='5', store_and_fwd_flag='N', pickup_datetime='2013-01-01 04:30:40', dropoff_datetime='2013-01-01 04:47:35', passenger_count='2', trip_time_in_secs='1014', trip_distance='7.10', pickup_longitude='-73.982719', pickup_latitude='40.767578', dropoff_longitude='-73.870903', dropoff_latitude='40.773895'),
 Row(medallion='36DD68427CE8A2F50FDAB5F22E6B6D73', hack_license='D09C75239B2C08468A14E68F6044C20B', vendor_id='CMT', rate_code='1', store_and_fwd_flag='N', pickup_datetime='2013-01-01 03:00:36', dropoff_datetime='2013-01-01 03:13:59', passenger_count='2', trip_time_in_secs='803', trip_distance='4.40', pickup_longitude='-74.009613', pickup_latitude='40.705166', dropoff_longitude='-73.981079', dropoff_latitude='40.741474')]

In [9]:
fare.take(2)

[Row(medallion='89D227B655E5C82AECF13C3F540D4CF4', hack_license='BA96DE419E711691B9445D6A6307C170', vendor_id='CMT', pickup_datetime='2013-01-01 15:11:48', payment_type='CSH', fare_amount='6.5', surcharge='0', mta_tax='0.5', tip_amount='0', tolls_amount='0', total_amount='7'),
 Row(medallion='0BD7C8F5BA12B88E0B67BED28BEA73D8', hack_license='9FD8F69F0804BDB5549F40E9DA1BE472', vendor_id='CMT', pickup_datetime='2013-01-06 00:18:35', payment_type='CSH', fare_amount='6', surcharge='0.5', mta_tax='0.5', tip_amount='0', tolls_amount='0', total_amount='7')]

## Drop Duplicates

Because there's some record that can't be correctly joined together. We need to drop them. <br/>
--- actually still keep some wrong record here because drop_dupicates will keep the first row.

In [10]:
fare = fare.drop_duplicates(subset=['medallion','hack_license','vendor_id','pickup_datetime'])
trip = trip.drop_duplicates(subset=['medallion','hack_license','vendor_id','pickup_datetime'])

In [11]:
print('rows of trip:',trip.count())
print('rows of fare:',fare.count())

rows of trip: 173177113
rows of fare: 173177113


## Merge Data & Check

In [12]:
# join the table on 'medallion','hack_license','vendor_id','pickup_datetime' so that each record would match exactly
joined_table = trip.join(fare,on = ['medallion','hack_license','vendor_id','pickup_datetime'])

In [13]:
print('rows of joined_table:',joined_table.count())

rows of joined_table: 173177113


In [15]:
joined_table.columns.__len__()

21

## Convert Fields to Corresponding Types

In [16]:
from pyspark.sql.types import *

In [17]:
# 'pickup_datetime' and 'dropoff_datetime' --> Timestamp
joined_table = joined_table.withColumn('pickup_datetime', joined_table['pickup_datetime'].cast(TimestampType()))
joined_table = joined_table.withColumn('dropoff_datetime', joined_table['dropoff_datetime'].cast(TimestampType()))

In [18]:
# 'passenger_count' and 'rate_code' --> Int
joined_table = joined_table.withColumn("passenger_count", joined_table["passenger_count"].cast(IntegerType()))
joined_table = joined_table.withColumn("rate_code", joined_table["rate_code"].cast(IntegerType()))

In [19]:
# other numeric fields --> Float
joined_table = joined_table.withColumn("trip_time_in_secs", joined_table["trip_time_in_secs"].cast(FloatType()))
joined_table = joined_table.withColumn("trip_distance", joined_table["trip_distance"].cast(FloatType()))
joined_table = joined_table.withColumn("pickup_longitude", joined_table["pickup_longitude"].cast(FloatType()))
joined_table = joined_table.withColumn("pickup_latitude", joined_table["pickup_latitude"].cast(FloatType()))
joined_table = joined_table.withColumn("dropoff_longitude", joined_table["dropoff_longitude"].cast(FloatType()))
joined_table = joined_table.withColumn("dropoff_latitude", joined_table["dropoff_latitude"].cast(FloatType()))
joined_table = joined_table.withColumn("fare_amount", joined_table["fare_amount"].cast(FloatType()))
joined_table = joined_table.withColumn("surcharge", joined_table["surcharge"].cast(FloatType()))
joined_table = joined_table.withColumn("mta_tax", joined_table["mta_tax"].cast(FloatType()))
joined_table = joined_table.withColumn("tip_amount", joined_table["tip_amount"].cast(FloatType()))
joined_table = joined_table.withColumn("tolls_amount", joined_table["tolls_amount"].cast(FloatType()))
joined_table = joined_table.withColumn("total_amount", joined_table["total_amount"].cast(FloatType()))

In [20]:
joined_table.printSchema()

root
 |-- medallion: string (nullable = true)
 |-- hack_license: string (nullable = true)
 |-- vendor_id: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- rate_code: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_time_in_secs: float (nullable = true)
 |-- trip_distance: float (nullable = true)
 |-- pickup_longitude: float (nullable = true)
 |-- pickup_latitude: float (nullable = true)
 |-- dropoff_longitude: float (nullable = true)
 |-- dropoff_latitude: float (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: float (nullable = true)
 |-- surcharge: float (nullable = true)
 |-- mta_tax: float (nullable = true)
 |-- tip_amount: float (nullable = true)
 |-- tolls_amount: float (nullable = true)
 |-- total_amount: float (nullable = true)



## In the following cells, please provide the requested code and output. Do not change the order and/or structure of the cells.

In the next cell, provide the code that saves your merged dataset to your S3 bucket.

In [21]:
joined_table.write.parquet('s3://weiteng-li/merged_data/')

In the next cell, print the schema of your merged dataset.

In [22]:
joined_table.printSchema()

root
 |-- medallion: string (nullable = true)
 |-- hack_license: string (nullable = true)
 |-- vendor_id: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- rate_code: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_time_in_secs: float (nullable = true)
 |-- trip_distance: float (nullable = true)
 |-- pickup_longitude: float (nullable = true)
 |-- pickup_latitude: float (nullable = true)
 |-- dropoff_longitude: float (nullable = true)
 |-- dropoff_latitude: float (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: float (nullable = true)
 |-- surcharge: float (nullable = true)
 |-- mta_tax: float (nullable = true)
 |-- tip_amount: float (nullable = true)
 |-- tolls_amount: float (nullable = true)
 |-- total_amount: float (nullable = true)



In the next cell, print the first 10 records of your merged dataset.

In [23]:
joined_table.show(10)

+--------------------+--------------------+---------+-------------------+---------+------------------+-------------------+---------------+-----------------+-------------+----------------+---------------+-----------------+----------------+------------+-----------+---------+-------+----------+------------+------------+
|           medallion|        hack_license|vendor_id|    pickup_datetime|rate_code|store_and_fwd_flag|   dropoff_datetime|passenger_count|trip_time_in_secs|trip_distance|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|payment_type|fare_amount|surcharge|mta_tax|tip_amount|tolls_amount|total_amount|
+--------------------+--------------------+---------+-------------------+---------+------------------+-------------------+---------------+-----------------+-------------+----------------+---------------+-----------------+----------------+------------+-----------+---------+-------+----------+------------+------------+
|00005007A9F30E289...|02015F5B7D1884620...|

In the next cell, print the row count of your merged dataset.

In [24]:
joined_table.count()

173177113

In [25]:
spark.stop()